In [2]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [3]:
!pip install tensorflow keras scikit-learn opencv-python


In [4]:
import os
import numpy as np
import cv2
from keras.preprocessing import image
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [5]:
# Loading and preprocessing data

def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)
        images.append(img_array)
        labels.append(label)
    return images, labels

In [6]:
folder_path_blight = r'C:\Users\52pun\Desktop\Smart_agri\Rice_disease_UCI\Bacterial_leaf_blight'
folder_path_brown_spot = r'C:\Users\52pun\Desktop\Smart_agri\Rice_disease_UCI\Brown_spot'
folder_path_leaf_smut = r'C:\Users\52pun\Desktop\Smart_agri\Rice_disease_UCI\Leaf_smut'

In [7]:
blight_images, blight_labels = load_images(folder_path_blight, 'Bacterial_leaf_blight')
brown_spot_images, brown_spot_labels = load_images(folder_path_brown_spot, 'Brown_spot')
leaf_smut_images, leaf_smut_labels = load_images(folder_path_leaf_smut, 'Leaf_smut')

In [8]:
X = np.concatenate([blight_images, brown_spot_images, leaf_smut_images], axis=0)
y = np.concatenate([blight_labels, brown_spot_labels, leaf_smut_labels], axis=0)

In [9]:
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Building the model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [18]:
#flatten
#Dense
#batch normalization
#dropout layer
#dense layer

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Training the Model

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 23s 7s/step - loss: 1.9991 - accuracy: 0.3026 - val_loss: 2.9668 - val_accuracy: 0.6500
Epoch 2/10
3/3 [==============================] - 18s 6s/step - loss: 0.3887 - accuracy: 0.8289 - val_loss: 2.9912 - val_accuracy: 0.6500
Epoch 3/10
3/3 [==============================] - 18s 6s/step - loss: 0.0681 - accuracy: 0.9868 - val_loss: 2.7219 - val_accuracy: 0.7000
Epoch 4/10
3/3 [==============================] - 17s 6s/step - loss: 0.1298 - accuracy: 0.9342 - val_loss: 2.5084 - val_accuracy: 0.7000
Epoch 5/10
3/3 [==============================] - 23s 9s/step - loss: 0.0319 - accuracy: 1.0000 - val_loss: 2.2859 - val_accuracy: 0.7000
Epoch 6/10
3/3 [==============================] - 23s 8s/step - loss: 0.0178 - accuracy: 1.0000 - val_loss: 2.1968 - val_accuracy: 0.6000
Epoch 7/10
3/3 [==============================] - 19s 6s/step - loss: 0.0293 - accuracy: 1.0000 - val_loss: 2.0504 - val_accuracy: 0.6000
Epoch 8/10
3/3 [==================

In [16]:
# Evaluating the Model

accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 5s 5s/step - loss: 0.1776 - accuracy: 0.9583
Test Accuracy: 95.83%


In [17]:
# Saving the model

model.save('rice_leaf_disease_model')